## Задание
Реализовать обучение линейной регрессии для задачи [boston house prices](https://www.kaggle.com/datasets/vikrishnan/boston-house-prices) с использованием pytorch.

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import autograd
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
print(torch.__version__)

import time

1.12.0+cu113


In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv('housing.csv', header = None, delimiter = r"\s+", names = column_names)
df.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
5,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222.0,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311.0,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311.0,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311.0,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311.0,15.2,386.71,17.10,18.9


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [ ]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [ ]:
df.isnull().any()

CRIM       False
ZN         False
INDUS      False
CHAS       False
NOX        False
RM         False
AGE        False
DIS        False
RAD        False
TAX        False
PTRATIO    False
B          False
LSTAT      False
MEDV       False
dtype: bool

In [ ]:
df.shape

(506, 14)

In [ ]:
df = df.apply(lambda x: (x - x.mean()) / x.std()) 
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,-0.419367,0.284548,-1.286636,-0.272329,-0.144075,0.413263,-0.119895,0.140075,-0.981871,-0.665949,-1.457558,0.440616,-1.074499,0.159528
1,-0.416927,-0.487240,-0.592794,-0.272329,-0.739530,0.194082,0.366803,0.556609,-0.867024,-0.986353,-0.302794,0.440616,-0.491953,-0.101424
2,-0.416929,-0.487240,-0.592794,-0.272329,-0.739530,1.281446,-0.265549,0.556609,-0.867024,-0.986353,-0.302794,0.396035,-1.207532,1.322937
3,-0.416338,-0.487240,-1.305586,-0.272329,-0.834458,1.015298,-0.809088,1.076671,-0.752178,-1.105022,0.112920,0.415751,-1.360171,1.181589
4,-0.412074,-0.487240,-1.305586,-0.272329,-0.834458,1.227362,-0.510674,1.076671,-0.752178,-1.105022,0.112920,0.440616,-1.025487,1.486032


In [ ]:
X = df.drop('MEDV', axis=1).to_numpy()
y = df['MEDV'].to_numpy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(354, 13) (152, 13) (354,) (152,)


In [ ]:
X_train = torch.tensor(X_train, dtype = torch.float32)
X_test = torch.tensor(X_test, dtype = torch.float32)
y_train = torch.tensor(y_train, dtype = torch.float32)
y_test = torch.tensor(y_test, dtype = torch.float32)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

torch.Size([354, 13]) torch.Size([152, 13]) torch.Size([354]) torch.Size([152])


In [ ]:
model = torch.nn.Sequential(torch.nn.Linear(X_train.shape[1], 1))

dataset = TensorDataset(X_train, y_train)

data_iter = DataLoader(dataset, batch_size = 10, shuffle = True)

for X, y in data_iter:
  print(X, y)
  break

tensor([[ 1.3907, -0.4872,  1.0150, -0.2723,  0.2184, -0.5104,  0.0862, -0.4211,
          1.6596,  1.5294,  0.8058,  0.1322,  0.7670],
        [ 1.1701, -0.4872,  1.0150, -0.2723,  1.5991, -0.4976,  0.6865, -0.9377,
          1.6596,  1.5294,  0.8058, -3.1516,  2.9921],
        [ 1.2564, -0.4872,  1.0150, -0.2723,  1.5991,  0.2510,  0.8784, -0.8512,
          1.6596,  1.5294,  0.8058, -3.6057,  0.7558],
        [-0.1527, -0.4872,  1.2307, -0.2723,  0.4341,  0.0489,  0.9778, -0.8050,
         -0.5225, -0.0311, -1.7347, -0.6527, -0.2175],
        [ 0.6656, -0.4872,  1.0150, -0.2723,  1.0727,  0.1357,  0.9601, -0.8676,
          1.6596,  1.5294,  0.8058, -3.2417,  1.6002],
        [ 0.4364, -0.4872,  1.0150, -0.2723,  1.0727, -0.1304,  0.3384, -0.8830,
          1.6596,  1.5294,  0.8058, -2.8473,  1.2417],
        [-0.2779, -0.4872,  1.2307, -0.2723,  0.4341,  0.9370,  1.0240, -0.9107,
         -0.5225, -0.0311, -1.7347,  0.0740, -1.1291],
        [ 0.9806, -0.4872,  1.0150, -0.2723,  0.

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001) 

In [ ]:
%%time
num_epoch = 30

for epoch in range(num_epoch):
    for x, y in data_iter:
        l = loss(model(x), y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print(f"epoch {epoch + 1}.......loss: {l.item()}")

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1.......loss: 0.9868783950805664
epoch 2.......loss: 1.1286300420761108
epoch 3.......loss: 0.2003074437379837
epoch 4.......loss: 0.25856882333755493
epoch 5.......loss: 0.188485249876976
epoch 6.......loss: 1.6742980480194092
epoch 7.......loss: 1.2826945781707764
epoch 8.......loss: 0.3135426342487335
epoch 9.......loss: 0.38933756947517395
epoch 10.......loss: 1.0498906373977661
epoch 11.......loss: 1.4824185371398926
epoch 12.......loss: 0.6382664442062378
epoch 13.......loss: 2.0448226928710938
epoch 14.......loss: 2.0837833881378174
epoch 15.......loss: 0.44188356399536133
epoch 16.......loss: 0.12554554641246796
epoch 17.......loss: 2.0335068702697754
epoch 18.......loss: 3.088805913925171
epoch 19.......loss: 0.16397586464881897
epoch 20.......loss: 3.099457263946533
epoch 21.......loss: 3.549363851547241
epoch 22.......loss: 1.350115418434143
epoch 23.......loss: 0.11513678729534149
epoch 24.......loss: 0.7418574690818787
epoch 25.......loss: 0.4878864288330078
epoch 26

In [ ]:
print(loss(model(X_test), y_test).item())

0.8433890342712402


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([152])) that is different to the input size (torch.Size([152, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
X_train.device

device(type='cpu')